In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cpu


In [11]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text =  f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [5]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode =  lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype =torch.long)
print(data[:100])

tensor([87, 31, 68, 71, 68, 73, 61, 78,  1, 54, 67, 57,  1, 73, 61, 58,  1, 50,
        62, 79, 54, 71, 57,  1, 62, 67,  1, 42, 79,  0,  0,  0,  1,  1, 28,  1,
        33, 54, 62, 73, 61, 59, 74, 65,  1, 45, 58, 56, 68, 71, 57,  1, 68, 59,
         1, 47, 61, 58, 62, 71,  1, 28, 66, 54, 79, 62, 67, 60,  1, 28, 57, 75,
        58, 67, 73, 74, 71, 58, 72,  0,  1,  1,  1,  1, 62, 67,  1, 54, 67,  1,
        48, 67, 57, 58, 71, 60, 71, 68, 74, 67])


In [9]:
n= int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) 
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([188310, 134383, 191161,  67329])
inputs:
tensor([[58, 57,  1, 55, 78,  1, 66, 78],
        [68, 66, 58, 73, 62, 66, 58, 72],
        [58,  0, 72, 73, 71, 62, 69, 58],
        [56, 54, 67,  1, 76, 58,  1, 60]])
targets:
tensor([[57,  1, 55, 78,  1, 66, 78,  1],
        [66, 58, 73, 62, 66, 58, 72,  1],
        [ 0, 72, 73, 71, 62, 69, 58, 72],
        [54, 67,  1, 76, 58,  1, 60, 68]])


In [14]:


x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target =y[t]
    print('when input is', context, 'target is', target)

when input is tensor([87]) target is tensor(31)
when input is tensor([87, 31]) target is tensor(68)
when input is tensor([87, 31, 68]) target is tensor(71)
when input is tensor([87, 31, 68, 71]) target is tensor(68)
when input is tensor([87, 31, 68, 71, 68]) target is tensor(73)
when input is tensor([87, 31, 68, 71, 68, 73]) target is tensor(61)
when input is tensor([87, 31, 68, 71, 68, 73, 61]) target is tensor(78)
when input is tensor([87, 31, 68, 71, 68, 73, 61, 78]) target is tensor(1)


In [ ]:
class BiagramLangugaeModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B,T, C = logits.shape
            logits = logits.view(B*T, C)
            targets =targets.view(B*T)
            loss =  F.cross_entropy(logits, targets)
        
        return  logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B,T) array of indicies in the current context
        for _ in range(max_new_tokens):
            #get the predicitions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits [:, -1, :] # becomes (B, C)
            # apply softmax to get probabilitie
            probs = F.softmax(logits, dim=-1) #(B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1)#(B,1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim =1) # (AB, T+1)
        return index

model = BiagramLangugaeModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype = torch.long, device =device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

        
    

In [ ]:
 a = torch.rand(2,3,5)
print(a.shape)
x,y,z = a.shape
a = a.view(x,y,z)
print(x,y,z)
print(a.shape)